In [89]:
import requests
import pandas as pd
import json

# Pull Latitude and Longitude from positionstack.com based on geographical user input.

In [214]:
# User input: Country
country_input = (input("Enter a Country using a 2-letter country code (Example: US): "))

Enter a Country using a 2-letter country code (Example: US): jp


In [215]:
# N/A: No need for this extra geographic variable.

# User input: State
# region_input = (input("Enter a region (City, County, State, etc.): "))

In [216]:
# User input: City

# HTTP GET Request Parameters:
# query	[Required] Specify your query as a free-text address, 
# place name or using any other common text-based location 
# identifier (e.g. postal code, city name, region name).

query_input = (input("Enter a location (City, Zip Code, etc.): "))

Enter a location (City, Zip Code, etc.): tokyo


In [217]:
# Geo reference for adding to final dataframe.
geo_input = country_input  + " | " + query_input
geo_input

'jp | tokyo'

In [218]:
import http.client, urllib.parse

conn = http.client.HTTPConnection('api.positionstack.com')

params = urllib.parse.urlencode({
    'access_key': '1fbf7de4ce05274e9b9005659970c429',
    
    # Variables get live user input above.
    'country': country_input,
#     'region': region_input,
    'query': query_input,
        
    # "limit" of 1 ensures only 1 Lat/Lon will be passed below.    
    'limit': 1
    })

conn.request('GET', '/v1/forward?{}'.format(params))

res = conn.getresponse()
geo = res.read()
geo_str = (geo.decode('utf-8'))

print(geo.decode('utf-8'))

###########################################################
# Still needs error handling.

{"data":[{"latitude":35.695126,"longitude":139.75391,"type":"locality","name":"Tokyo","number":null,"postal_code":null,"street":null,"confidence":1,"region":"Tokyo","region_code":"TK","county":"Chiyoda","locality":"Tokyo","administrative_area":null,"neighbourhood":null,"country":"Japan","country_code":"JPN","continent":"Asia","label":"Tokyo, Japan"}]}


In [219]:
# Help from AskBCS:
# Result was one long string, that only looked 
# like a dictionary, but wasn't.
# Use "type()" to identify whether or not 
# result is actually a dict.

type(geo_str)

str

In [220]:
# Convert string to json (which is a dictionary).
# geo_json
geo_json = json.loads(geo_str)

print(geo_json)

{'data': [{'latitude': 35.695126, 'longitude': 139.75391, 'type': 'locality', 'name': 'Tokyo', 'number': None, 'postal_code': None, 'street': None, 'confidence': 1, 'region': 'Tokyo', 'region_code': 'TK', 'county': 'Chiyoda', 'locality': 'Tokyo', 'administrative_area': None, 'neighbourhood': None, 'country': 'Japan', 'country_code': 'JPN', 'continent': 'Asia', 'label': 'Tokyo, Japan'}]}


##### Reminder: Double parentheses are functional in the for loops.

In [221]:
# # geo_json_coord = lat and lon combined into a tuple.

# geo_json_coord = geo_json['data']
# latlon_tup = []
# for x in geo_json_coord:
#     latlon_tup.append((x['latitude'], x['longitude']))

In [222]:
# Pull latitude value by itself.

geo_json_lat = geo_json['data']
var_lat = []
for x in geo_json_lat:
    var_lat.append((x['latitude']))
    
var_lat[0]

35.695126

In [223]:
# Pull longitude value by itself.

geo_json_lon = geo_json['data']
var_lon = []
for x in geo_json_lon:
    var_lon.append((x['longitude']))
    
var_lon[0]

139.75391

# Push Latitude and Longitude to Weather API URL, based on start date and stop date from user input.

## Hourly data.

In [224]:
# List of columns, for starter dataset, hourly:

# hourly_columns = ['time',
#              'temperature_2m_degF', 
#              'relativehumidity_2m_perc', 
#              'rain_inch',
#              'snowfall_cm', 
#              'cloudcover_perc', 
#              'windspeed_10m_mph',
#              'winddirection_10m_deg']

#  We pulled data from 2000-01-01 to 2020-12-31.

Original Open Meteo API URL, hourly:

https://archive-api.open-meteo.com/v1/era5?latitude=52.52&longitude=13.41&start_date=2000-01-01&end_date=2020-12-31&hourly=temperature_2m,relativehumidity_2m,rain,snowfall,cloudcover,windspeed_10m,winddirection_10m&timezone=America%2FLos_Angeles&temperature_unit=fahrenheit&windspeed_unit=mph&precipitation_unit=inch

In [225]:
# User input: Start Date
# start_date_input = (input("Enter the start date (YYYY-MM-DD): "))

In [226]:
# User input: End Date
# end_date_input = (input("Enter the end date (YYYY-MM-DD): "))

In [227]:
# Break URL into snippets to be assembled below, hourly:

url_snip_hr_1 = "https://archive-api.open-meteo.com/v1/era5?latitude=" 
url_snip_hr_2 = str(var_lat[0])   # latitude, input from previous json
url_snip_hr_3 = "&longitude="
url_snip_hr_4 = str(var_lon[0])   # longitude, input from previous json



url_snip_hr_5 = "&start_date="
# url_snip_hr_6 = start_date_input   # start date, input from user
url_snip_hr_6 = "2010-01-01"   # --- OR --- static start date, for dummy data

url_snip_hr_7 = "&end_date="
# url_snip_hr_8 = end_date_input   # end date, input from user
url_snip_hr_8 = "2019-12-31"   # --- OR ---  static end date, for dummy data



url_snip_hr_9 = "&hourly="   # elements, hourly
url_snip_hr_10 = "temperature_2m,"   # data element
url_snip_hr_11 = "relativehumidity_2m,"   # data element
url_snip_hr_12 = "rain,"   # data element
url_snip_hr_13 = "snowfall,"   # data element
url_snip_hr_14 = "cloudcover,"   # data element
url_snip_hr_15 = "windspeed_10m,"   # data element
url_snip_hr_16 = "winddirection_10m"   # data element

url_snip_hr_17 = "&timezone=America%2FLos_Angeles"   # time zone
url_snip_hr_18 = "&temperature_unit=fahrenheit"   # temp unit
url_snip_hr_19 = "&windspeed_unit=mph"   # windspeed unit
url_snip_hr_20 = "&precipitation_unit=inch"   # precip unit

In [228]:
# Contatenate the url snippets, hourly.

url_hr_concat_full = \
    url_snip_hr_1 + \
    url_snip_hr_2 + \
    url_snip_hr_3 + \
    url_snip_hr_4 + \
    url_snip_hr_5 + \
    url_snip_hr_6 + \
    url_snip_hr_7 + \
    url_snip_hr_8 + \
    url_snip_hr_9 + \
    url_snip_hr_10 + \
    url_snip_hr_11 + \
    url_snip_hr_12 + \
    url_snip_hr_13 + \
    url_snip_hr_14 + \
    url_snip_hr_15 + \
    url_snip_hr_16 + \
    url_snip_hr_17 + \
    url_snip_hr_18 + \
    url_snip_hr_19 + \
    url_snip_hr_20

In [229]:
print(url_hr_concat_full)

https://archive-api.open-meteo.com/v1/era5?latitude=35.695126&longitude=139.75391&start_date=2010-01-01&end_date=2019-12-31&hourly=temperature_2m,relativehumidity_2m,rain,snowfall,cloudcover,windspeed_10m,winddirection_10m&timezone=America%2FLos_Angeles&temperature_unit=fahrenheit&windspeed_unit=mph&precipitation_unit=inch


In [230]:
# Data comes in as one long string.
weather_hr_str = requests.get(url_hr_concat_full).text

In [231]:
# Convert string to dictionary.
weather_hr_json = json.loads(weather_hr_str)

In [232]:
# Verify that data type is dictionary.
type(weather_hr_json)

dict

In [233]:
# Convert json dictionary into dataframe.

# Standard "pd.json_normalize(weather_json)" doesn't work 
# with these nested levels. Also tried adding "record_path"
# and "meta" tags, but still didn't work.
# Found "from_records" alternative that does work.

# N/A: Code from:
# https://stackoverflow.com/questions/66894920/json-normalise-from-nested-api-to-df-throwing-type-error
# Creates "pivot" table format with all entries as columns, 
# all variables as rows.
# Not what we need, but might be useful for other datasets.
# dftest3 = pd.DataFrame.from_records(weather_json['hourly']).T.reset_index()


# Code from:
# https://stackoverflow.com/questions/66894920/json-normalise-from-nested-api-to-df-throwing-type-error

weather_hourly_raw = pd.DataFrame.from_records(weather_hr_json['hourly'])
weather_hourly_raw

,cloudcover,rain,relativehumidity_2m,snowfall,temperature_2m,time,winddirection_10m,windspeed_10m
0,1,0.0,37,0.0,45.0,2010-01-01T00:00,15.0,8.6
1,0,0.0,48,0.0,41.5,2010-01-01T01:00,21.0,8.1
2,0,0.0,54,0.0,39.3,2010-01-01T02:00,16.0,7.4
3,4,0.0,62,0.0,35.8,2010-01-01T03:00,24.0,3.9
4,1,0.0,64,0.0,34.9,2010-01-01T04:00,20.0,5.2
...,...,...,...,...,...,...,...,...
87643,24,0.0,41,0.0,43.2,2019-12-31T19:00,5.0,4.7
87644,35,0.0,40,0.0,44.8,2019-12-31T20:00,27.0,2.0
87645,8,0.0,40,0.0,46.0,2019-12-31T21:00,120.0,1.8
87646,0,0.0,41,0.0,46.8,2019-12-31T22:00,156.0,2.7


In [234]:
# Add geo input from user as new column, for reference.

weather_hourly_raw['geo'] = geo_input
weather_hourly_raw

,cloudcover,rain,relativehumidity_2m,snowfall,temperature_2m,time,winddirection_10m,windspeed_10m,geo
0,1,0.0,37,0.0,45.0,2010-01-01T00:00,15.0,8.6,jp | tokyo
1,0,0.0,48,0.0,41.5,2010-01-01T01:00,21.0,8.1,jp | tokyo
2,0,0.0,54,0.0,39.3,2010-01-01T02:00,16.0,7.4,jp | tokyo
3,4,0.0,62,0.0,35.8,2010-01-01T03:00,24.0,3.9,jp | tokyo
4,1,0.0,64,0.0,34.9,2010-01-01T04:00,20.0,5.2,jp | tokyo
...,...,...,...,...,...,...,...,...,...
87643,24,0.0,41,0.0,43.2,2019-12-31T19:00,5.0,4.7,jp | tokyo
87644,35,0.0,40,0.0,44.8,2019-12-31T20:00,27.0,2.0,jp | tokyo
87645,8,0.0,40,0.0,46.0,2019-12-31T21:00,120.0,1.8,jp | tokyo
87646,0,0.0,41,0.0,46.8,2019-12-31T22:00,156.0,2.7,jp | tokyo


## Daily data.

In [235]:
# List of columns, for starter dataset, daily:

# daily_columns = ['time',
#            'temperature_2m_degF_max',
#            'temperature_2m_degF_min',  
#            'rain_inch',
#            'snowfall_cm',
#            'precipitation_hours'
#           ]

#  We pulled data from 2000-01-01 to 2020-12-31.

Original Open Meteo API URL, daily:

https://archive-api.open-meteo.com/v1/era5?latitude=52.52&longitude=13.41&start_date=2000-01-01&end_date=2020-12-31&daily=temperature_2m_max,temperature_2m_min,rain_sum,snowfall_sum,precipitation_hours&timezone=America%2FLos_Angeles&temperature_unit=fahrenheit&windspeed_unit=mph&precipitation_unit=inch

In [236]:
# Break URL into snippets to be assembled below, daily:

url_snip_dy_1 = "https://archive-api.open-meteo.com/v1/era5?latitude=" 
url_snip_dy_2 = str(var_lat[0])   # latitude, input from previous json
url_snip_dy_3 = "&longitude="
url_snip_dy_4 = str(var_lon[0])   # longitude, input from previous json



url_snip_dy_5 = "&start_date="
# url_snip_dy_6 = start_date_input   # start date, input from user
url_snip_dy_6 = "2010-01-01"   # --- OR --- static start date, for dummy data

url_snip_dy_7 = "&end_date="
# url_snip_dy_8 = end_date_input   # end date, input from user
url_snip_dy_8 = "2019-12-31"   # --- OR ---  static end date, for dummy data



url_snip_dy_9 = "&daily="   # elements, daily
url_snip_dy_10 = "temperature_2m_max,"   # data element
url_snip_dy_11 = "temperature_2m_min,"   # data element
url_snip_dy_12 = "rain_sum,"   # data element
url_snip_dy_13 = "snowfall_sum,"   # data element
url_snip_dy_14 = "precipitation_hours"   # data element


url_snip_dy_15 = "&timezone=America%2FLos_Angeles"   # time zone
url_snip_dy_16 = "&temperature_unit=fahrenheit"   # temp unit
url_snip_dy_17 = "&windspeed_unit=mph"   # windspeed unit
url_snip_dy_18 = "&precipitation_unit=inch"   # precip unit

In [237]:
# Contatenate the url snippets, hourly.

url_dy_concat_full = \
    url_snip_dy_1 + \
    url_snip_dy_2 + \
    url_snip_dy_3 + \
    url_snip_dy_4 + \
    url_snip_dy_5 + \
    url_snip_dy_6 + \
    url_snip_dy_7 + \
    url_snip_dy_8 + \
    url_snip_dy_9 + \
    url_snip_dy_10 + \
    url_snip_dy_11 + \
    url_snip_dy_12 + \
    url_snip_dy_13 + \
    url_snip_dy_14 + \
    url_snip_dy_15 + \
    url_snip_dy_16 + \
    url_snip_dy_17 + \
    url_snip_dy_18

In [238]:
print(url_dy_concat_full)

https://archive-api.open-meteo.com/v1/era5?latitude=35.695126&longitude=139.75391&start_date=2010-01-01&end_date=2019-12-31&daily=temperature_2m_max,temperature_2m_min,rain_sum,snowfall_sum,precipitation_hours&timezone=America%2FLos_Angeles&temperature_unit=fahrenheit&windspeed_unit=mph&precipitation_unit=inch


In [239]:
# Data comes in as one long string.
weather_dy_str = requests.get(url_dy_concat_full).text

In [240]:
# Convert string to dictionary.
weather_dy_json = json.loads(weather_dy_str)

In [241]:
# Verify that data type is dictionary.
type(weather_dy_json)

dict

In [242]:
# Convert json dictionary into dataframe.

# Standard "pd.json_normalize(weather_json)" doesn't work 
# with these nested levels. Also tried adding "record_path"
# and "meta" tags, but still didn't work.
# Found "from_records" alternative that does work.

# N/A: Code from:
# https://stackoverflow.com/questions/66894920/json-normalise-from-nested-api-to-df-throwing-type-error
# Creates "pivot" table format with all entries as columns, 
# all variables as rows.
# Not what we need, but might be useful for other datasets.
# dftest3 = pd.DataFrame.from_records(weather_json['hourly']).T.reset_index()


# Code from:
# https://stackoverflow.com/questions/66894920/json-normalise-from-nested-api-to-df-throwing-type-error

weather_daily_raw = pd.DataFrame.from_records(weather_dy_json['daily'])
weather_daily_raw

,precipitation_hours,rain_sum,snowfall_sum,temperature_2m_max,temperature_2m_min,time
0,0.0,0.00,0.0,50.9,32.2,2010-01-01
1,0.0,0.00,0.0,49.3,32.0,2010-01-02
2,0.0,0.00,0.0,47.0,33.6,2010-01-03
3,3.0,0.05,0.0,54.5,38.1,2010-01-04
4,1.0,0.00,0.0,51.4,35.9,2010-01-05
...,...,...,...,...,...,...
3647,0.0,0.00,0.0,50.6,37.8,2019-12-27
3648,0.0,0.00,0.0,49.3,36.0,2019-12-28
3649,2.0,0.01,0.0,48.6,41.7,2019-12-29
3650,0.0,0.00,0.0,64.1,41.3,2019-12-30


In [243]:
# Add geo input from user as new column, for reference.

weather_daily_raw['geo'] = geo_input
weather_daily_raw

,precipitation_hours,rain_sum,snowfall_sum,temperature_2m_max,temperature_2m_min,time,geo
0,0.0,0.00,0.0,50.9,32.2,2010-01-01,jp | tokyo
1,0.0,0.00,0.0,49.3,32.0,2010-01-02,jp | tokyo
2,0.0,0.00,0.0,47.0,33.6,2010-01-03,jp | tokyo
3,3.0,0.05,0.0,54.5,38.1,2010-01-04,jp | tokyo
4,1.0,0.00,0.0,51.4,35.9,2010-01-05,jp | tokyo
...,...,...,...,...,...,...,...
3647,0.0,0.00,0.0,50.6,37.8,2019-12-27,jp | tokyo
3648,0.0,0.00,0.0,49.3,36.0,2019-12-28,jp | tokyo
3649,2.0,0.01,0.0,48.6,41.7,2019-12-29,jp | tokyo
3650,0.0,0.00,0.0,64.1,41.3,2019-12-30,jp | tokyo


In [246]:
# Hourly ##############################

# San Francisco
# New York
# London
# Berlin
# Tokyo

# sf_df_hourly
# ny_df_hourly
# london_df_hourly
# berlin_df_hourly
# tokyo_df_hourly

# Start Date: 2010-01-01
# End Date: 2019-12-31

# sf_df_hourly = weather_hourly_raw
# sf_df_hourly

# dummy5cities_hourly = pd.concat([sf_df_hourly, ny_df_hourly, london_df_hourly, berlin_df_hourly, tokyo_df_hourly])

# dummy5cities_hourly.to_csv('../Resources/dummy5cities_hourly.csv')




# Daily ##############################

# sf_df_daily
# ny_df_daily
# london_df_daily
# berlin_df_daily
# tokyo_df_daily


# tokyo_df_daily = weather_daily_raw
# tokyo_df_daily


# dummy5cities_daily = pd.concat([sf_df_daily, ny_df_daily, london_df_daily, berlin_df_daily, tokyo_df_daily])
# dummy5cities_daily

# dummy5cities_daily.to_csv('../Resources/dummy5cities_daily.csv')